In [ ]:
import glob
import os

from astropy.io import fits
import numpy as np
import numpy.ma as ma
import pandas as pd

In [ ]:
working_directory = '/Users/cmartlin/Desktop/'
today = '12-09-2022'
postflash_data = pd.read_pickle('../../2022_data/Feb_2022_flc_all_stats_postflash.pkl')

In [ ]:
def stack(list_of_files,out_file,error_file):
    hdr = fits.getheader(list_of_files[0], 1)
    nx = hdr['NAXIS1']
    ny = hdr['NAXIS2']
    nf = len(list_of_files)
    data_array_1 = np.empty((nf, ny, nx), dtype=float)
    data_array_2 = np.empty((nf, ny, nx), dtype=float)
    set_data=fits.getdata(list_of_files[0], 1)
    rms_1 = np.zeros(len(list_of_files), dtype=float)
    rms_2 = np.zeros(len(list_of_files), dtype=float)
    error_array_1 = np.empty((nf, ny, nx), dtype=float)
    error_array_2 = np.empty((nf, ny, nx), dtype=float)
    total_error_1 = np.zeros_like(set_data, dtype=float)
    total_error_2 = np.zeros_like(set_data, dtype=float)
    for i , f in enumerate(list_of_files):
        data_1=fits.getdata(f, 1)
        data_2=fits.getdata(f, 4)
        error_1=fits.getdata(f, 2)
        error_2=fits.getdata(f, 5)
        DQ_1=fits.getdata(f, 3)
        DQ_2=fits.getdata(f, 6)
        #data_1=data1
        #data_2=data2
        mask_1=np.zeros_like(data_1, dtype=bool)
        mask_2=np.zeros_like(data_2, dtype=bool)
        mask_1[DQ_1>=2**13]= True
        mask_2[DQ_2>=2**13]= True
        error_1[(0<DQ_1 & (DQ_1<2**13))]=0.00001
        error_2[(0<DQ_2& (DQ_2<2**13))]=0.00001
        error_1_sq=error_1**2
        error_2_sq=error_2**2
        masked_data_1= ma.array(data=data_1, mask=mask_1)
        masked_data_2= ma.array(data=data_2, mask=mask_2)
        data_array_1[i, :, :] = masked_data_1
        rms_1[i] = masked_data_1.std()
        data_array_2[i, :, :] = masked_data_2
        rms_2[i] = masked_data_2.std()
        total_error_1=total_error_1+(error_1_sq)
        total_error_2=total_error_2+(error_2_sq)
    sr_total_error_1=np.sqrt(total_error_1)
    sr_total_error_2=np.sqrt(total_error_2)
    fin_error_1=(sr_total_error_1/(float(len(list_of_files))))
    fin_error_2=(sr_total_error_2/(float(len(list_of_files))))
    image_median_1 = np.median(data_array_1, axis=0)
    image_median_2 = np.median(data_array_2, axis=0)
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(image_median_1))
    new_hdul.append(fits.ImageHDU(image_median_2))
    new_hdul.writeto(out_file, overwrite=True)
    #error
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(fin_error_1))
    new_hdul.append(fits.ImageHDU(fin_error_2))
    new_hdul.writeto(error_file,overwrite=True)

In [ ]:
def create_reference_file(year, working_directory, today, cadence):
    fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
    shutters = ('A', 'B')
    for shutter in shutters: 
        fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == '{}'.format(shutter)) & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)] 
        fullframe_pf_year = fullframe_pf[(fullframe_pf['datetime'] > '{}-01-01 00:00:00'.format(str(year))) & (fullframe_pf['datetime'] < '{}-01-01 00:00:00'.format(str(year+cadence)))]
        paths_year = fullframe_pf_year.path.tolist()
        print(len(paths_year))
        outfile_year = '{}{}_cadence{}_fullframe_{}_flc_stack_{}.fits'.format(working_directory,str(year), str(cadence), shutter, today)
        error_outfile_year = '{}{}_cadence{}_fullframe_{}_flc_error_stack_{}.fits'.format(working_directory,str(year), str(cadence), shutter, today)
        print(outfile_year)

    return paths_year, outfile_year, error_outfile_year

To run the stacking code you need to run: stack(path_list, path_outfile, path_error_outfile)

path_list - this is the list you create from the path column of the subset of the pandas database. 

path_outfile - this is the path and filename you want for your outfile. If you just put "outfile.fits" the file will save where you are running the notebook. I suggest start with the path to wfc3u so we keep them in a central location and to have the file be descriptive. Ex: "/grp/hst/wfc3u/postflash_2021/2020_stack_flt.fits"

path_error_outfile - this is the path and filename you want for your outfile of the calcuated error. If you just put "outfile.fits" the file will save where you are running the notebook. I suggest start with the path to wfc3u so we keep them in a central location and to have the file be descriptive. Ex: "/grp/hst/wfc3u/postflash_2021/2020_stack_error_flt.fits"


Below I show how to create the proper cut for a single year of data from the pandas database which has already been cut to Fullframe, Medium current, Shutter A. 

From there I update the path_list, outfile, and error_outfile to be specfic to the year being tested. 

Once that is run, I use those parameters to run the `stack` function. Outputs will be saved to '/grp/hst/wfc3u/postflash_2021/'. 

I then update a copy of a new set to 2013 and run `stack` on that set. 

The following years/groups of years can be done the same way. Following the creation of all the stacked files we will still need to calculate the mean/median of the stacked images to plot them over time.

bi-yearly creation

In [ ]:
years = [2012, 2014, 2016, 2018, 2020]
cadence = 2

for y in years: 
    paths_year, outfile_year, error_outfile_year = create_reference_file(y, working_directory, today, cadence)
    #stack(paths_year, outfile_year, error_outfile_year)

5 year grouping

In [ ]:
years = [2012,2017]
cadence = 5

for y in years: 
    paths_year, outfile_year, error_outfile_year = create_reference_file(y, working_directory, today, cadence)
    #stack(paths_year, outfile_year, error_outfile_year)

All years grouping: 

In [ ]:
years = [2012]
cadence = 10

for y in years: 
    paths_year, outfile_year, error_outfile_year = create_reference_file(y, working_directory, today, cadence)
    #stack(paths_year, outfile_year, error_outfile_year)

Adding a change_permissions function: 

In [ ]:
def change_permissions(path_to_files):
    """Change permissions of the output files and directories. 
        Code structure borrowed from cal_uvis_make_darks/FileIO.py. 
        Parameters
        ----------
            path_to_files : str
                Path to the directory that contains the files you want
                to update. 
    """
    
    os.chdir(path_to_files)
    all_directories = glob.glob('*')

    for directory in all_directories:
        try:
            os.chmod(directory, 0o775)
        except: 
            print('Could not change permissions.')
        for root, subdirs, files in os.walk(directory):
            for subdir in subdirs:
                try:
                    os.chmod(os.path.join(root, subdir), 0o775)
                except: 
                    print('Could not change permissions.')
            for name in files:
                try:
                    os.chmod(os.path.join(root, name), 0o775)
                except: 
                    print('Could not change permissions.')

In [ ]:
change_permissions(working_directory)